In [1]:
import re
import pandas as pd
import os
import itertools
import json
import sklearn
import evaluate

/home/michele.maggini/miniconda3/envs/bert_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
language = "Arabic"

In [8]:
###Dataset###
tsv_directory = '/home/michele.maggini/dataset/CLEF_1C/CLEF_1c'
filename = f'CLEF_1C_{language}.tsv'
df_path = os.path.join(tsv_directory, filename)
df_gt = pd.read_csv(df_path, sep="\t", engine="python")

cd = '/home/michele.maggini/Llama3/'
filename = f'CLEF1C_{language}.csv'
path = os.path.join(cd, filename)
df_ans = pd.read_csv(path)

In [9]:
df_gt

,tweet_text,class_label
0,دخلت اسجل لقاح بمستشفى الجامعه وبيومين صار كل ...,0
1,تعرّف على أول دولة ستحصل على اللقاح الروسي ضد ...,0
2,البروفيسورة سارة جيلبرت قائدة فريق العمل في لق...,0
3,#الصين :اللقاح المضاد ل #فيروس_كورونا سيكون مت...,0
4,وزير الصحة الروسى: تطعيم الأطباء بلقاح #كورونا...,0
...,...,...
4949,الأجسام المضادة S309 ستساعد في اكتشاف علاج أو ...,0
4950,مع انتظار سيطول لايجاد لقاح ل #فيروس_كورونا كث...,0
4951,نداء للتفكر لا ينبغي أن نتعاقد مع أي مورد لقاح...,1
4952,#روسيا بين تاريخين مجيدين فلاديمير بوتين يعلن ...,0


In [10]:
for index, row in df_ans.iterrows():
    print(row)

0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 0, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 1, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 2, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 3, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 4, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 5, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 6, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 7, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 8, dtype: object
0    1\n\nHere's the step-by-step analysis:\n\n**Ma...
Name: 9, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 10, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 11, dtype: object
0    0\n\nReasoning:\n\n1. Main keywords and phrase...
Name: 1

In [11]:
def extract_last_line(text):
    # Split the text into lines
    lines = text.strip().split('\n')
    # Return the last line
    return lines[-1]

In [12]:
listLastline = df_ans['0'].apply(lambda x: extract_last_line(x))
df_lastline = listLastline.to_list()
len(df_lastline)

4954

In [13]:
def remove_keywords(input_list):
    # Define the keywords to be removed
    keywords = ["(True)", "(true)", "(False)", "(false)"]
    
    # Function to remove keywords from a single string
    def remove_from_string(s):
        for keyword in keywords:
            s = s.replace(keyword, "")
        return s.strip()  # Remove leading/trailing whitespace
    
    # Apply the function to each element in the list
    output_list = [remove_from_string(item) for item in input_list]
    
    return output_list

In [14]:
output_list = remove_keywords(df_lastline)
len(output_list)
output_list

['Overall, the tweet does not appear to be harmful or offensive, and therefore, I would rate it as 0.',
 'Overall, the tweet appears to be a neutral and informative message about the COVID-19 pandemic, without any harmful or offensive content.',
 'Overall, the tweet does not appear to be harmful or damaging to society, and therefore, I would rate it as 0.',
 'Overall, the tweet does not appear to be harmful or dangerous to the community, and therefore, I would rate it as 0.',
 'Overall, the tweet appears to be a factual and informative message about a public health issue, and does not contain any harmful or offensive content.',
 'Overall, the tweet appears to be a neutral and informative announcement about a potential breakthrough in the fight against COVID-19, without any harmful or offensive content.',
 'Overall, the tweet is spreading misinformation and promoting anti-vaccination sentiments, which can be harmful to public health. However, it does not contain any explicit threats, ha

In [34]:
def extract_integers(input_list):
    # This will store the extracted integers
    integers = []
    # This will store the index and sentences with more than one integer and those without keywords
    special_cases = {}

    # Define a regex pattern to match integers
    integer_pattern = re.compile(r'\b\d+\b')
    
    for index, item in enumerate(input_list):
        # Remove "COVID-19" from the item
        item = item.replace("COVID-19", "")
        
        # Find all integers in the modified item
        found_integers = integer_pattern.findall(item)
        
        if found_integers:
            if len(found_integers) > 1:
                special_cases[index] = item
                integers.append(2)
            else:
                # Convert found integers to int
                found_integers = list(map(int, found_integers))
                # Add the found integers to the result list
                integers.extend(found_integers)
        else:
            # Check for keywords if no integers are found
            item_lower = item.lower()
            if 'true' in item_lower or 'accurate' in item_lower or 'factual' in item_lower:
                integers.append(0)
            elif 'false' in item_lower:
                integers.append(1)
            else:
                # Store the index and sentence if neither integers nor keywords are found
                integers.append(2)
                special_cases[index] = item
    
    return integers, special_cases


In [35]:
labels_parsed, special_cases = extract_integers(output_list)
len(labels_parsed), len(special_cases)

(4954, 2114)

In [36]:
special_cases

{1: 'Overall, the tweet appears to be a neutral and informative message about the  pandemic, without any harmful or offensive content.',
 5: 'Overall, the tweet appears to be a neutral and informative announcement about a potential breakthrough in the fight against , without any harmful or offensive content.',
 7: 'Overall, the conversation appears to be a lighthearted and informal discussion between a retired Russian diplomat and a Twitter user, discussing the effectiveness of Russian vaccines and the preference for natural remedies. The tone is friendly and playful, and there is no evidence of harm or aggression.',
 8: 'Overall, the tweet appears to be a legitimate and informative message about the  pandemic, and does not contain any harmful or offensive content.',
 9: 'Overall, I would classify this tweet as having a negative impact on society, as it contains language that is aggressive, divisive, and potentially threatening.',
 13: 'Overall, this tweet is a clear example of hate sp

In [37]:
labels_parsed

[0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 1,
 2,
 2,
 1,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 21,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 1,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 0,
 1,
 2,
 0,
 0,
 1,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,

In [38]:
df_gt

,tweet_text,class_label
0,دخلت اسجل لقاح بمستشفى الجامعه وبيومين صار كل ...,0
1,تعرّف على أول دولة ستحصل على اللقاح الروسي ضد ...,0
2,البروفيسورة سارة جيلبرت قائدة فريق العمل في لق...,0
3,#الصين :اللقاح المضاد ل #فيروس_كورونا سيكون مت...,0
4,وزير الصحة الروسى: تطعيم الأطباء بلقاح #كورونا...,0
...,...,...
4949,الأجسام المضادة S309 ستساعد في اكتشاف علاج أو ...,0
4950,مع انتظار سيطول لايجاد لقاح ل #فيروس_كورونا كث...,0
4951,نداء للتفكر لا ينبغي أن نتعاقد مع أي مورد لقاح...,1
4952,#روسيا بين تاريخين مجيدين فلاديمير بوتين يعلن ...,0


In [39]:

gold_ans = df_gt['class_label'].tolist()
print("gold_ans:", gold_ans)


gold_ans: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,

In [40]:
k = [el for el in labels_parsed]
print(k)

[0, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 0, 1, 0, 0, 1, 2, 2, 1, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 21, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 1, 1, 2, 2, 2, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 1, 0, 0, 0, 0, 2, 0, 1, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 2, 2, 1, 2, 2, 0, 1, 2, 0, 0, 1, 1, 2, 0, 0, 2, 2, 2, 2, 0, 1, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 0, 1957, 0, 0, 2, 0, 0, 2, 2, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0,

In [41]:
def compute_metrics(predictions, labels):
        accuracy_metric = evaluate.load("accuracy")
        precision_metric = evaluate.load("precision")
        recall_metric = evaluate.load("recall")
        f1_metric = evaluate.load("f1")


        accuracy = accuracy_metric.compute(predictions=predictions, references=labels)[
            "accuracy"
        ]
        precision = precision_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["precision"]
        recall = recall_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["recall"]
        f1 = f1_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["f1"]

        return {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1-score": f1,
        }


results=compute_metrics(labels_parsed, gold_ans)
print(f"FNNet, CoT, prompt with temperature=0.1:\n", json.dumps(results, indent=4))

FNNet, CoT, prompt with temperature=0.1:
 {
    "accuracy": 0.47719014937424303,
    "precision": 0.8190340784815995,
    "recall": 0.47719014937424303,
    "f1-score": 0.6008758957592103
}


/home/michele.maggini/miniconda3/envs/bert_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
